In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import cv2
import numpy as np

In [2]:
# Defina o caminho para o arquivo .pth do modelo treinado
device = torch.device("cuda")

model = models.resnet18(pretrained=True)
num_features = model.fc.in_features # extraindo features da camada fc
model.fc = nn.Linear(num_features, 7) # é numero de classes

model.load_state_dict(torch.load('model.pth'))
model.to(device)
model.eval()

/home/arthur/anaconda3/envs/skincancer/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/arthur/anaconda3/envs/skincancer/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
# Carregue a imagem e faça o pré-processamento
image = Image.open('nv.jpeg')
# Aplicar transformações à imagem para prepará-la para o modelo
preprocess = transforms.Compose([
    transforms.Resize(256),      # Redimensione para 256x256 pixels
    transforms.CenterCrop(224),  # Realize um corte central de 224x224 pixels
    transforms.ToTensor(),       # Converta a imagem em um tensor
    transforms.Normalize(        # Normalize os valores dos canais RGB
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Aplicar as transformações à imagem
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # Adicione uma dimensão de lote (batch)

In [4]:
# Certifique-se de que o modelo e o tensor de entrada estejam na mesma GPU, se disponível
if torch.cuda.is_available():
    model = model.cuda()
    input_batch = input_batch.cuda()

In [5]:
# Faça a previsão
with torch.no_grad():
    output = model(input_batch)

# Aplicar a função softmax para obter probabilidades
probabilities = torch.softmax(output, dim=1)

# Nome das classes
class_names = ['Doença de Bowen','Carcinoma basocelular','Ceratose benignas','Dermatofibroma','Melanoma','Nevos melanocíticos','Lesões vasculares']

# Obtenha as probabilidades para cada classe
class_probabilities = {class_names[i]: prob.item() * 100 for i, prob in enumerate(probabilities[0])}

# Ordene as probabilidades em ordem decrescente
sorted_class_probabilities = dict(sorted(class_probabilities.items(), key=lambda x: x[1], reverse=True))

# Exemplo de como acessar as probabilidades por classe
for class_name, prob_percentage in sorted_class_probabilities.items():
    print(f"{class_name}: {prob_percentage:.2f}%")

Nevos melanocíticos: 99.10%
Melanoma: 0.83%
Doença de Bowen: 0.07%
Dermatofibroma: 0.00%
Lesões vasculares: 0.00%
Carcinoma basocelular: 0.00%
Ceratose benignas: 0.00%


In [6]:
sorted_class_probabilities

{'Nevos melanocíticos': 99.09728765487671,
 'Melanoma': 0.8293876424431801,
 'Doença de Bowen': 0.07326069171540439,
 'Dermatofibroma': 5.3303540425986284e-05,
 'Lesões vasculares': 1.5480109638588146e-06,
 'Carcinoma basocelular': 1.3144658694841382e-07,
 'Ceratose benignas': 6.566189649295784e-11}

In [ ]:
import torch
import cv2
import numpy as np

# Calcule a ativação da camada convolucional desejada
# Neste exemplo, usaremos a camada 'layer4[1].conv1' como no seu código anterior
activation = model.layer4[1].conv1.weight
activation = activation[0].unsqueeze(0)  # Adicione uma dimensão de lote (batch)

# Redimensione a ativação para o tamanho da imagem original
activation = activation.detach().cpu().numpy()  # Converta para um array NumPy
activation -= activation.min()
activation /= activation.max()
activation *= 255
activation = np.uint8(activation)

# Redimensione a ativação para o tamanho da imagem original
activation = cv2.resize(activation[0], (input_tensor.shape[2], input_tensor.shape[1]))

# Aplique um mapa de cores para torná-lo visualmente mais informativo
heatmap = cv2.applyColorMap(activation, cv2.COLORMAP_JET)

# Misture a imagem original com o mapa de calor
input_image = input_batch.squeeze(0).permute(1, 2, 0).cpu().numpy() * 255
input_image = np.uint8(input_image)
superimposed_img = cv2.addWeighted(input_image, 0.7, heatmap, 0.3, 0)

# Salve a imagem com o mapa de calor destacado
cv2.imwrite('./img/img.jpg', superimposed_img)

True